Kimberly Adams 
DSC 530 September 2022

Download data files from github

In [1]:
from os.path import basename, exists


def download(url):
    filename = basename(url)
    if not exists(filename):
        from urllib.request import urlretrieve

        local, _ = urlretrieve(url, filename)
        print("Downloaded " + local)

# Download author's functions.
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkstats2.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/thinkplot.py")

In [2]:
# Import numpy and pandas packages
import numpy as np
import pandas as pd

# Import author's functions
import thinkstats2
import thinkplot

In [3]:
# Import NSFG data
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/nsfg.py")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/first.py")

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dct")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemPreg.dat.gz")

download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dct")
download("https://github.com/AllenDowney/ThinkStats2/raw/master/code/2002FemResp.dat.gz")

# Import into dataframes
import first

live, firsts, others = first.MakeFrames()

In [4]:
# Import StatsModels for multiple regression
import statsmodels.formula.api as smf

# Limit data set to live births that occured after week 30
live = live[live.prglngth>30]

In [5]:
# Import author's function to summarize the results important stats
def SummarizeResults(results):
    """Prints the most important parts of linear regression results:
    results: RegressionResults object
    """
    for name, param in results.params.items():
        pvalue = results.pvalues[name]
        print('%s   %0.3g   (%.3g)' % (name, param, pvalue))

    try:
        print('R^2 %.4g' % results.rsquared)
        ys = results.model.endog
        print('Std(ys) %.4g' % ys.std())
        print('Std(res) %.4g' % results.resid.std())
    except AttributeError:
        print('R^2 %.4g' % results.prsquared)

In [6]:
# Import patsy and re functions
import patsy
import re


# Import author's functions
# Find r2 values for ALL variables
def GoMining(df):
    """Searches for variables that predict birth weight.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            # Modified to look for pregnancy length predictors that work best with pregnancy order
            formula = 'prglngth ~ pregordr + ' + name
            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError, patsy.PatsyError) as e:
            continue
        
        variables.append((results.rsquared, name))

    return variables


def ReadVariables():
    """Reads Stata dictionary files for NSFG data.

    returns: DataFrame that maps variables names to descriptions
    """
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars


def MiningReport(variables, n=30):
    """Prints variables with the highest R^2.

    t: list of (R^2, variable name) pairs
    n: number of pairs to print
    """
    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name, r2)

# Chapter 11
### Excercise 11-1

Suppose one of your co-workers is expecting a baby and you are participating in an office pool to predict the date of birth. Assuming that bets are placed during the 30th week of pregnancy, what variables could you use to make the best prediction? You should limit yourself to variables that are known before the birth, and likely to be available to the people in the pool.

*Looking through the list of variables in the dataset, here is my short list of possibly helpful variables:*
- pregordr = what pregnancy is this for the mother?
- babysex = Boy or girl?
- agepreg = age at pregnancy outcome
- race = mother's race

Dependent variable:
- prglngth = duration of completed pregnancy in weeks


*Let's see if we can find which ones are good predictors.*

In [7]:
print('Pregnancy length as predicted by pregnancy number')
formula = 'prglngth ~ pregordr'

# Combines the dataframe and the selected variables and computes ordinary least squares
model = smf.ols(formula, data=live)

# Fit finds a model that fits the data and outputs a regression object
results = model.fit()

# Outputs abreviated summary statistics from the fitted regression
SummarizeResults(results)

Pregnancy length as predicted by pregnancy number
Intercept   39   (0)
pregordr   -0.033   (0.0187)
R^2 0.0006222
Std(ys) 1.898
Std(res) 1.897


*Pregnancy number is a good predictor of pregnancy length (p-value 0.0187)*

In [8]:
print('Pregnancy length as predicted by baby gender')
formula = 'prglngth ~ babysex'

# Combines the dataframe and the selected variables and computes ordinary least squares
model = smf.ols(formula, data=live)

# Fit finds a model that fits the data and outputs a regression object
results = model.fit()

# Outputs abreviated summary statistics from the fitted regression
SummarizeResults(results)

Pregnancy length as predicted by baby gender
Intercept   39   (0)
babysex   -0.0525   (0.193)
R^2 0.0001912
Std(ys) 1.898
Std(res) 1.898


*Baby's gender is NOT a good predictor of pregnancy length (p-value 0.193)*

In [9]:
print('Pregnancy length as predicted by mother age')
formula = 'prglngth ~ agepreg'

# Combines the dataframe and the selected variables and computes ordinary least squares
model = smf.ols(formula, data=live)

# Fit finds a model that fits the data and outputs a regression object
results = model.fit()

# Outputs abreviated summary statistics from the fitted regression
SummarizeResults(results)

Pregnancy length as predicted by mother age
Intercept   38.9   (0)
agepreg   -0.00143   (0.693)
R^2 1.76e-05
Std(ys) 1.898
Std(res) 1.898


*Mother's age is NOT a good predictor of pregnancy length (p-value 0.693)*

In [10]:
print('Pregnancy length as predicted by race')
formula = 'prglngth ~ race'

# Combines the dataframe and the selected variables and computes ordinary least squares
model = smf.ols(formula, data=live)

# Fit finds a model that fits the data and outputs a regression object
results = model.fit()

# Outputs abreviated summary statistics from the fitted regression
SummarizeResults(results)

Pregnancy length as predicted by race
Intercept   38.8   (0)
race   0.0594   (0.0947)
R^2 0.0003144
Std(ys) 1.898
Std(res) 1.898


*Mother's race is an ok predictor of pregnancy length (p-value 0.0947)*

*So from all that we have a short list of pregnancy number (pregordr) and mother's race (race).*

*Let's go data mining and see what we find...*

In [11]:
# Import data into live and resp dataframes and then join them based on caseid
import nsfg

# Filter for pregnancies longer than 30 weeks and create dataframe
live = live[live.prglngth>30]

# Create dataframe of responses
resp = nsfg.ReadFemResp()

# Define identifier
resp.index = resp.caseid

# Join dataframs based on shared identifier
join = live.join(resp, on='caseid', rsuffix='_r')

In [12]:
variables = GoMining(join)
MiningReport(variables)

prglngth 1.0 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.8063083681530956 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
totalwgt_lb 0.12532496173938545
birthwgt_lb 0.12062744546420201 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.1041933580414558 LOW BIRTHWEIGHT - BABY 1
mosgest 0.096042142512203 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i 0.023315315330877828 PRGLNGTH IMPUTATION FLAG
datcon01_i 0.006266940713901659 DATCON01 IMPUTATION FLAG
canhaver 0.006165339208669018 DF-1 PHYSICALLY DIFFICULT FOR R TO HAVE A BABY
con1mar1_i 0.00595401826330233 CON1MAR1 IMPUTATION FLAG
nbrnaliv 0.005062293351614122 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
chedmarn 0.004364138596498979 CB-11 HIGHEST LEVEL OF EDUCATION-1ST HUSBAND
ptsb4mar 0.003845006562791742 CH-3 # OF MALE SEX PARTNERS PRIOR TO 1ST MARRIAGE
monsx1233 0.003807137277237027 EC-8 SEX IN SEPTEMBER 2002?
mar1con1_i 0.0036301023456278836 MAR1CON1 IMPUTATION FLAG
methstop01 0.

/var/folders/fn/gx6f152n3ml122r2lwdkncvc0000gn/T/ipykernel_78772/773936657.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_vars = vars1.append(vars2)


*From our data mining, the top distinct predictor of pregnancy length when combined with pregnancy order are birth weight (birthwgt_lb) at R2=0.12 and number of births a multiple birth pregnancy (nbrnaliv) (R2=0.005).*

In [13]:
# Run the model to see stats
model = smf.ols('prglngth ~ pregordr + birthwgt_lb + nbrnaliv', data=live)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.122
Model:                            OLS   Adj. R-squared:                  0.122
Method:                 Least Squares   F-statistic:                     407.9
Date:                Thu, 27 Oct 2022   Prob (F-statistic):          3.61e-248
Time:                        22:00:00   Log-Likelihood:                -17618.
No. Observations:                8825   AIC:                         3.524e+04
Df Residuals:                    8821   BIC:                         3.527e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      35.8109      0.160    223.358      0.000      35.497      36.125
pregordr       -0.0367      0.013     -2.764      0.006      -0.063      -0.011
birthwgt_lb     0.5105      0.015     34.235      0.000       0.481       0.540
nbrnaliv       -0.3752      0.108     -3.484      0.000      -0.586      -0.164
==============================================================================
Omnibus:                     1052.152   Durbin-Watson:                   1.616
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4742.660
Skew:                          -0.508   Prob(JB):                         0.00
Kurtosis:                       6.444   Cond. No.                         72.4
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

*We can't actually run the model for an estimate without more information, in this case is this her first pregnancy, how much does the baby weigh now, and how many infants is she carrying?*

*It is also worth noting that the R2 of the model is 0.122 which is not great and indicates that the model as is would not be great at predicting.*

### Excercise 11-3

If the quantity you want to predict is a count, you can use Poisson regression, which is implemented in StatsModels with a function called `poisson`. It works the same way as `ols` and `logit`. As an exercise, let’s use it to predict how many children a woman has born; in the NSFG dataset, this variable is called `numbabes`.

Suppose you meet a woman who is 35 years old, black, and a college graduate whose annual household income exceeds $75,000. How many children would you predict she has born?

In [14]:
# Add non-linear model for age
join['age2'] = join.age_r**2

formula = 'numbabes ~ agepreg + age2 + C(race) + totincr + educat'
model = smf.poisson(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.690116
         Iterations 7


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8877
Method:                           MLE   Df Model:                            6
Date:                Thu, 27 Oct 2022   Pseudo R-squ.:                 0.02933
Time:                        22:00:01   Log-Likelihood:                -15015.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                9.249e-193
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.4021      0.039     35.930      0.000       1.326       1.479
C(race)[T.2]    -0.1439      0.015     -9.661      0.000      -0.173      -0.115
C(race)[T.3]    -0.0953      0.025     -3.867      0.000      -0.144      -0.047
agepreg          0.0027      0.001      2.001      0.045    5.47e-05       0.005
age2             0.0003   1.62e-05     17.061      0.000       0.000       0.000
totincr         -0.0176      0.002     -9.247      0.000      -0.021      -0.014
educat          -0.0451      0.003    -14.890      0.000      -0.051      -0.039
================================================================================
"""

In [15]:
# Apply input values
columns = ['agepreg', 'age2', 'race', 'totincr', 'educat']
# Age = 35, Race = black=1, Income = $75,000=14, Education = college grad=16
new = pd.DataFrame([[35, 35**2, 1, 14, 16]], columns=columns)
results.predict(new)

0    2.379328
dtype: float64

*Based on my model, I would predict she would have 2.3 children, but since that is a decimal number, then let's say 2-3 children.*

### Excercise 11-4

If the quantity you want to predict is categorical, you can use multinomial logistic regression, which is implemented in StatsModels with a function called mnlogit. As an exercise, let’s use it to guess whether a woman is married, cohabitating, widowed, divorced, separated, or never married; in the NSFG dataset, marital status is encoded in a variable called rmarital. 

Suppose you meet a woman who is 25 years old, white, and a high school graduate whose annual household income is about $45,000. What is the probability that she is married, cohabitating, etc?

In [16]:
formula='rmarital ~ agepreg + age2 + C(race) + totincr + educat'
model = smf.mnlogit(formula, data=join)
results = model.fit()
results.summary() 

Optimization terminated successfully.
         Current function value: 1.080914
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8849
Method:                           MLE   Df Model:                           30
Date:                Thu, 27 Oct 2022   Pseudo R-squ.:                  0.1707
Time:                        22:00:02   Log-Likelihood:                -9602.8
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.1160      0.258     15.960      0.000       3.610       4.621
C(race)[T.2]    -0.8495      0.089     -9.538      0.000      -1.024      -0.675
C(race)[T.3]    -0.5716      0.136     -4.208      0.000      -0.838      -0.305
agepreg         -0.0527      0.009     -6.151      0.000      -0.069      -0.036
age2            -0.0006   9.85e-05     -5.706      0.000      -0.001      -0.000
totincr         -0.1299      0.012    -11.256      0.000      -0.152      -0.107
educat          -0.1781      0.019     -9.282      0.000      -0.216      -0.140
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.9517      0.726     -2.690      0.007      -3.374      -0.529
C(race)[T.2]    -0.3785      0.237     -1.595      0.111      -0.844       0.087
C(race)[T.3]     0.0947      0.336      0.282      0.778      -0.563       0.753
agepreg         -0.0385      0.018     -2.111      0.035      -0.074      -0.003
age2             0.0021      0.000      7.665      0.000       0.002       0.003
totincr         -0.3247      0.032    -10.172      0.000      -0.387      -0.262
educat          -0.0783      0.049     -1.585      0.113      -0.175       0.018
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -0.7939      0.251     -3.166      0.002      -1.285      -0.302
C(race)[T.2]    -0.2210      0.094     -2.340      0.019      -0.406      -0.036
C(race)[T.3]    -0.6871      0.172     -3.999      0.000      -1.024      -0.350
agepreg         -0.0719      0.008     -9.140      0.000      -0.087      -0.056
age2             0.0012   9.71e-05     12.418      0.000       0.001       0.001
totincr         -0.2240      0.012    -19.215      0.000      -0.247      -0.201
educat           0.1131      0.018      6.427      0.000       0.079       0.148
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.7639      0.308      5.724      0.000       1.160       2.368
C(race)[T.2]    -0.9680      0.104     -9.272      0.000      -1.173      -0.763
C(race)[T.3]    -0.5024      0.157     -3.209      0.001      -0.809      -0.196
agepreg         -0.0705      0.010     -7.094      0.000      -0.090      -0.051
age2             0.0004      0.000      3.275      0.001       0.000       0.001
totincr         -0.2900      0.015    -19.937      0.000      -0.319      -0.261
educat           0.0258      0.022      1.163      0.245      -0.018       0.069
-

In [17]:
# Apply input values
columns = ['agepreg', 'age2', 'race', 'totincr', 'educat']
# Age = 25, Race = white=2, Income = $45,000=11, Education = high school grad=12
new = pd.DataFrame([[25, 25**2, 2, 11, 12]], columns=columns)
results.predict(new)

,0,1,2,3,4,5
0,0.779053,0.108925,0.001152,0.032901,0.02099,0.056979


KEY
- 0 CURRENTLY MARRIED
- 1 NOT MARRIED BUT LIVING WITH OPP SEX PARTNER
- 2 WIDOWED
- 3 DIVORCED
- 4 SEPARATED FOR REASONS OF MARITAL DISCORD
- 5 NEVER BEEN MARRIED

*The value under each category label is the probability of that category on a scale of 0-1. For example, based on this model the probablity that she is married is 77.9%.*